In [22]:
import pandas as pd
import numpy as np
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import spacy
import itertools as it
import os

In [ ]:
import nltk
from gensim import corpora
from collections import defaultdict
from pprint import pprint
import codecs

In [ ]:
pd.set_option('display.max_colwidth', 100)

In [4]:
nlp = spacy.load('en') 
## python -m spacy download en

In [5]:
review = pd.read_csv("/Volumes/Transcend/MDS_Yelp/yelp_review.csv")

In [6]:
nv_reviews = review.sample(5000)
nv_reviews['parsed_text'] = None
nv_reviews.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,parsed_text
1155267,s9GBl-WBsCJrl49llUfTig,psaaZ3Ulc3EFGv37UpxxRg,SOoNoV7_YJcuWgVfetspfg,2,2015-08-04,I would be giving zero stars if my second experience was my only experience. On this second occa...,5,1,0,None
3007981,NMz3UJtl4ruL39QQGDv5ZQ,TudUls_yRpkZoDe9m9O_tg,7svPiqFz7ad82VcdDbBcJw,5,2017-07-04,"I love this Starbucks, nice peeps and the service is outstanding.\nThey know their business that...",0,0,0,None
1921077,6fNzrwuJyUnDtR8BcqNRfQ,5MChqlj0L3DzVi5-cS_xvg,CZKHXlDuy3IagC2W881fyA,5,2017-12-11,So I've never been big on cocktails. I know they're good but i never really raves about how good...,0,0,0,None
788957,4YU22XKLI9z8oHOIur-c9w,iQbqQwF6ttvdMbpbqEGIaQ,onoZsIFPJXumpV_ndxcGTQ,2,2007-04-08,"When I first came to town, I was stuck going here for my internet access.\n\nI had to pay for a ...",2,1,0,None
274477,NbNhfEJoqLcqnz6dfwHeTQ,lSjdYb2wussdNmj5XvN45Q,01fuY2NNscttoTxOYbuZXw,5,2015-06-10,I have only had the opportunity to eat here a couple times but it is on the top of the list for ...,0,1,0,None


In [7]:
type(nv_reviews['text'].values[0])

str

In [8]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [9]:
documents = nv_reviews['text'].tolist()

In [10]:
docs = []
for i in range(len(documents)):
    docs.append(nlp(documents[i]))

In [11]:
documents[0]

'I would be giving zero stars if my second experience was my only experience. On this second occasion I had barely walked in when told "the guy that cuts your kind of hair isn\'t here, come back tomorrow." \n\nMy haircut is so simple that any barber can do it. Any. Same length all the way around with a little extra left on the front top. No fades, no special shapes cut into the side of my head. Real simple. \n\nThe heavyset man there didn\'t ask what style I wanted, just presumed black guy wants a black guy haircut. I was coming here because it was closer to my place than my regular place, Nate\'s Barber Shop on 7th Ave. \n\nThank you for reminding me why it\'s worth it to me to go to my regular place and stay the loyal customer I am. I\'ll never go back to Steve\'s Barber Shop. \n\nLest anyone think I\'m implying racism, I\'m not. You can come to your own conclusions. I\'m invoking Hanlon\'s Razor: never presume malice where stupidity is more likely.'

In [12]:
docs[0]

I would be giving zero stars if my second experience was my only experience. On this second occasion I had barely walked in when told "the guy that cuts your kind of hair isn't here, come back tomorrow." 

My haircut is so simple that any barber can do it. Any. Same length all the way around with a little extra left on the front top. No fades, no special shapes cut into the side of my head. Real simple. 

The heavyset man there didn't ask what style I wanted, just presumed black guy wants a black guy haircut. I was coming here because it was closer to my place than my regular place, Nate's Barber Shop on 7th Ave. 

Thank you for reminding me why it's worth it to me to go to my regular place and stay the loyal customer I am. I'll never go back to Steve's Barber Shop. 

Lest anyone think I'm implying racism, I'm not. You can come to your own conclusions. I'm invoking Hanlon's Razor: never presume malice where stupidity is more likely.

In [13]:
token_lemma = [token.lemma_ for token in docs[1]]

In [14]:
unigram_review = [token.lemma_ for token in docs[0] if not punct_space(token)]

In [15]:
# remove any remaining stopwords
unigram_review = [term for term in unigram_review if term not in spacy.lang.en.STOP_WORDS]
# nlp.Defaults.stop_words works as well

In [16]:
unigram_review

['-PRON-',
 'zero',
 'star',
 '-PRON-',
 'second',
 'experience',
 '-PRON-',
 'experience',
 'second',
 'occasion',
 '-PRON-',
 'barely',
 'walk',
 'tell',
 'guy',
 'cut',
 '-PRON-',
 'kind',
 'hair',
 'come',
 'tomorrow',
 '-PRON-',
 'haircut',
 'simple',
 'barber',
 '-PRON-',
 'length',
 'way',
 'little',
 'extra',
 'left',
 'fade',
 'special',
 'shape',
 'cut',
 '-PRON-',
 'head',
 'real',
 'simple',
 'heavyset',
 'man',
 'ask',
 'style',
 '-PRON-',
 'want',
 'presume',
 'black',
 'guy',
 'want',
 'black',
 'guy',
 'haircut',
 '-PRON-',
 'come',
 '-PRON-',
 'close',
 '-PRON-',
 'place',
 '-PRON-',
 'regular',
 'place',
 'nate',
 "'s",
 'barber',
 'shop',
 '7th',
 'ave',
 'thank',
 '-PRON-',
 'remind',
 '-PRON-',
 '-PRON-',
 'worth',
 '-PRON-',
 '-PRON-',
 '-PRON-',
 'regular',
 'place',
 'stay',
 'loyal',
 'customer',
 '-PRON-',
 '-PRON-',
 'steve',
 "'s",
 'barber',
 'shop',
 'lest',
 'think',
 '-PRON-',
 'imply',
 'racism',
 '-PRON-',
 '-PRON-',
 'come',
 '-PRON-',
 'conclusion',


In [17]:
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser

sentences = Text8Corpus(datapath('testcorpus.txt'))

In [18]:
sentences

In [19]:
datapath('testcorpus.txt')

'/anaconda3/lib/python3.6/site-packages/gensim/test/test_data/testcorpus.txt'

In [20]:
open('/anaconda3/lib/python3.6/site-packages/gensim/test/test_data/testcorpus.txt')

<_io.TextIOWrapper name='/anaconda3/lib/python3.6/site-packages/gensim/test/test_data/testcorpus.txt' mode='r' encoding='UTF-8'>

In [21]:
phrases = Phrases(sentences, min_count=1, threshold=1)
phrases
#https://stackoverflow.com/questions/35716121/how-to-extract-phrases-from-corpus-using-gensim

In [22]:
phrases[[u'trees', u'graph', u'minors']]

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['trees_graph', 'minors']

In [23]:
nv_reviews = nv_reviews.reset_index(drop=True)
nv_reviews.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,parsed_text
0,s9GBl-WBsCJrl49llUfTig,psaaZ3Ulc3EFGv37UpxxRg,SOoNoV7_YJcuWgVfetspfg,2,2015-08-04,I would be giving zero stars if my second experience was my only experience. On this second occa...,5,1,0,None
1,NMz3UJtl4ruL39QQGDv5ZQ,TudUls_yRpkZoDe9m9O_tg,7svPiqFz7ad82VcdDbBcJw,5,2017-07-04,"I love this Starbucks, nice peeps and the service is outstanding.\nThey know their business that...",0,0,0,None
2,6fNzrwuJyUnDtR8BcqNRfQ,5MChqlj0L3DzVi5-cS_xvg,CZKHXlDuy3IagC2W881fyA,5,2017-12-11,So I've never been big on cocktails. I know they're good but i never really raves about how good...,0,0,0,None
3,4YU22XKLI9z8oHOIur-c9w,iQbqQwF6ttvdMbpbqEGIaQ,onoZsIFPJXumpV_ndxcGTQ,2,2007-04-08,"When I first came to town, I was stuck going here for my internet access.\n\nI had to pay for a ...",2,1,0,None
4,NbNhfEJoqLcqnz6dfwHeTQ,lSjdYb2wussdNmj5XvN45Q,01fuY2NNscttoTxOYbuZXw,5,2015-06-10,I have only had the opportunity to eat here a couple times but it is on the top of the list for ...,0,1,0,None


In [24]:
review_txt_filepath = os.path.join('review_text_all.txt')

review_count = 0

# create & open a new file in write mode
with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:
    # if this review is not about a restaurant, skip to the next one
    # write the restaurant review as a line in the new file
    # escape newline characters in the original review text
    for i in range(len(nv_reviews['text'])):
        review_txt_file.write(nv_reviews['text'][i].replace('\n', '\\n') + '\n')
        review_count += 1

print(u'''Text from {:,} restaurant reviews written to the new txt file.'''.format(review_count))

Text from 5,000 restaurant reviews written to the new txt file.


In [25]:
nv_reviews['text']

0       I would be giving zero stars if my second experience was my only experience. On this second occa...
1       I love this Starbucks, nice peeps and the service is outstanding.\nThey know their business that...
2       So I've never been big on cocktails. I know they're good but i never really raves about how good...
3       When I first came to town, I was stuck going here for my internet access.\n\nI had to pay for a ...
4       I have only had the opportunity to eat here a couple times but it is on the top of the list for ...
5       Best Thai food! Always fresh and consistent. Food is always hot and made to order. Lunch special...
6       We absolutely love Delux - especially the sweet potato fries and the Delux Burger.  It's not unc...
7       Dr Budd and his team are the best. You will never regret getting orthodontic care by him. He is ...
8       Excellent calamari and Cal Italia pizza!! Sat in the sports bar area.  Conveniently located by F...
9       I was so happy I got

In [26]:
os.getcwd()

'/Users/rajeshpothamsetty/Sagitta/Reviewly'

In [27]:
# filepaths# filepat 
unigram_sentences_filepath = os.path.join('unigram.txt')
bigram_sentences_filepath = os.path.join('bigrams.txt')
bigram_model_filepath = os.path.join('bigram_model_all')
trigram_sentences_filepath = os.path.join('trigrams.txt')
trigram_model_filepath = os.path.join('trigram_model_all')

In [28]:
#unigrams
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        f.write(sentence + '\n')

unigram_sentences = LineSentence(unigram_sentences_filepath)



In [29]:
data1 = range(10)

# This creates a NEW list
data1[2:5]

# This creates an iterator that iterates over the EXISTING list
for x in it.islice(data1, 2, 5):
    print(x)

2
3
4


In [30]:
%%time
use_model = 'no'
# this is a bit time consuming - make the if statement True# this is 
# if you want to execute modeling yourself.
if use_model == 'no':
    bigram_model = Phrases(unigram_sentences)
    bigram_model.save(bigram_model_filepath)
else:    
    # load the finished model from disk
    bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 1.43 s, sys: 54.6 ms, total: 1.49 s
Wall time: 1.49 s


In [31]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if use_model == 'no':
    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for unigram_sentence in unigram_sentences:
            bigram_sentence = u' '.join(bigram_model[unigram_sentence]) 
            f.write(bigram_sentence + '\n')

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 3.34 s, sys: 59.1 ms, total: 3.4 s
Wall time: 3.48 s


In [32]:
bigram_sentences  = LineSentence(bigram_sentences_filepath)

In [33]:
%%time

use_model = 'no'
# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if use_model == 'no':
    trigram_model = Phrases(bigram_sentences)
    trigram_model.save(trigram_model_filepath)
else:    
    # load the finished model from disk
    trigram_model = Phrases.load(trigram_model_filepath)

CPU times: user 1.44 s, sys: 59.2 ms, total: 1.5 s
Wall time: 1.58 s


In [34]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if use_model == 'no':
    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:  
        for bigram_sentence in bigram_sentences:        
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])       
            f.write(trigram_sentence + '\n')

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 3.2 s, sys: 41.5 ms, total: 3.24 s
Wall time: 3.27 s


In [35]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [36]:
trigram_reviews_filepath  = os.path.join('trigram_transformed_reviews_all.txt')

In [37]:
%%time

use_model_trigram = 'no'
# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if use_model_trigram == 'no':
    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        for parsed_review in nlp.pipe(line_review(review_txt_filepath), batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review if term not in spacy.lang.en.STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 2min 58s, sys: 48 s, total: 3min 46s
Wall time: 2min 22s


In [38]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 10, 11):
    print(review)
    print(type(review))

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 10, 11):
        temp = review
        print(review)
        print(type(review))

Original:

My husband and I were so taken with the beauty of the live edge wood. We had a coffee table top made and they even attached our mesquite legs to the top. It is so beautiful. Bjorn made the process so simple. I have been down there many times now and they always show the same interest in what you want to do.  They also do commercial work and some of their projects are just amazing. I plan to go to some of their DIY workshops. Terry Schuman

<class 'str'>
----

Transformed:

-PRON- husband -PRON- beauty live edge wood -PRON- coffee table -PRON- attach -PRON- mesquite leg -PRON- beautiful bjorn process simple -PRON- many_time -PRON- interest -PRON- want -PRON- commercial work -PRON- project amazing -PRON- plan -PRON- diy workshop terry schuman

<class 'str'>


In [39]:
temp

'-PRON- husband -PRON- beauty live edge wood -PRON- coffee table -PRON- attach -PRON- mesquite leg -PRON- beautiful bjorn process simple -PRON- many_time -PRON- interest -PRON- want -PRON- commercial work -PRON- project amazing -PRON- plan -PRON- diy workshop terry schuman\n'

In [40]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [41]:
# from linkedin course
# not a great combination with tri-gram
# should try independently
x_temp = temp.replace('_', " ")
clean_text(temp) 
# remove 'pron'
# add spell Checker
# 

['pron',
 'husband',
 'pron',
 'beauti',
 'live',
 'edg',
 'wood',
 'pron',
 'coffe',
 'tabl',
 'pron',
 'attach',
 'pron',
 'mesquit',
 'leg',
 'pron',
 'beauti',
 'bjorn',
 'process',
 'simpl',
 'pron',
 'manytim',
 'pron',
 'interest',
 'pron',
 'want',
 'pron',
 'commerci',
 'work',
 'pron',
 'project',
 'amaz',
 'pron',
 'plan',
 'pron',
 'diy',
 'workshop',
 'terri',
 'schuman',
 '']

In [42]:
clean_text(x_temp)

['pron',
 'husband',
 'pron',
 'beauti',
 'live',
 'edg',
 'wood',
 'pron',
 'coffe',
 'tabl',
 'pron',
 'attach',
 'pron',
 'mesquit',
 'leg',
 'pron',
 'beauti',
 'bjorn',
 'process',
 'simpl',
 'pron',
 'mani',
 'time',
 'pron',
 'interest',
 'pron',
 'want',
 'pron',
 'commerci',
 'work',
 'pron',
 'project',
 'amaz',
 'pron',
 'plan',
 'pron',
 'diy',
 'workshop',
 'terri',
 'schuman',
 '']

In [43]:
acac = open('trigram_transformed_reviews_all.txt', "r")
lines = list(acac)
lines_ = pd.Series(lines)

In [44]:
lines_

0       -PRON- zero_star -PRON- second experience -PRON- experience second occasion -PRON- barely walk_i...
1       -PRON- love starbucks nice peep service outstanding -PRON- know -PRON- business sure -PRON- -PRO...
2       -PRON- big cocktail -PRON- know -PRON- good rave_about good -PRON- come brickyard downtown -PRON...
3       -PRON- come town -PRON- stick -PRON- internet access -PRON- pay monthly subscription use wired c...
4       -PRON- opportunity eat_here couple time -PRON- list place go_back last_time -PRON- -PRON- pimp'n...
5       good thai food fresh consistent food hot order lunch_special $ 6.95 come choice rice soup salad ...
6       -PRON- absolutely_love delux especially sweet_potato_fry delux burger -PRON- uncommon -PRON- com...
7       dr budd -PRON- team good -PRON- will_never regret orthodontic care -PRON- -PRON- patient kind -P...
8       excellent calamari cal italia pizza sat sport_bar area conveniently_locate fremont street experi...
9       -PRON- happy -PRON- 

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(lines_)
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names())

(5000, 17114)
['', '0', '000', '002', '007', '02142015', '025', '047ounc', '06', '060', '062316', '071615', '080', '09', '0951', '0jrucdobl4pqsig2gfnon3lz37s2wh0khkrvmwbvm', '0star', '1', '10', '100', '1000', '10000', '100000', '1000000x', '1000x', '100139', '100c', '100calori', '100stm', '100th', '1010', '1015', '1015minut', '1016', '102', '1020', '10200', '1027', '1028pm', '103', '1030', '1030am', '1030pm', '1040', '1040am', '1045', '1045am', '1045pm', '105', '1050', '106', '107thmcdowel', '108', '10800', '109', '1095', '1099', '10am', '10mile', '10min', '10minut', '10minutebefor', '10minutelat', '10month', '10per', '10pm', '10th', '10x', '10year', '10yo', '11', '110', '1100', '11042017', '110person', '111', '1110', '11152011', '11216', '11272017', '113', '1130', '1130am', '114', '1140am', '1145', '1145am', '115', '1150', '1150am', '1155', '11616the', '117', '11746', '119', '1195', '1199', '11am', '11ish', '11min', '11pm', '12', '120', '1200', '1205', '120ish', '1210', '1212015', '12

In [46]:
data_sample = lines_[0:20]

tfidf_vect_sample = TfidfVectorizer(analyzer=clean_text)
X_tfidf_sample = tfidf_vect_sample.fit_transform(data_sample)
print(X_tfidf_sample.shape)
print(tfidf_vect_sample.get_feature_names())

X_tfidf_df = pd.DataFrame(X_tfidf_sample.toarray())
X_tfidf_df.columns = tfidf_vect_sample.get_feature_names()
X_tfidf_df

(20, 534)
['', '10', '1000', '2', '2star', '3', '5', '5star', '695', '7th', 'abit', 'ableto', 'about10minut', 'absolutelylov', 'access', 'accommod', 'actual', 'add', 'alright', 'amaz', 'anyway', 'appetit', 'appoint', 'appreci', 'area', 'arjuna', 'artisan', 'asian', 'ask', 'astonishingli', 'aswel', 'atmospher', 'attach', 'attent', 'audienc', 'authent', 'avail', 'ave', 'averag', 'aw', 'awesom', 'babbl', 'bacon', 'bad', 'badexperi', 'bag', 'bahn', 'band', 'bar', 'barber', 'barbershop', 'bare', 'bartend', 'basebal', 'basket', 'beauti', 'beef', 'beer', 'begin', 'best', 'bhel', 'big', 'bite', 'bjorn', 'black', 'bland', 'bomb', 'bowl', 'bread', 'brickyard', 'budd', 'burger', 'burrito', 'busi', 'buy', 'cafe', 'cal', 'calamari', 'cannot', 'capit', 'captiv', 'car', 'care', 'careabout', 'carrot', 'centrallyloc', 'chain', 'chanc', 'chandler', 'check', 'cheesi', 'chicken', 'chile', 'chipotl', 'chocolatecak', 'choic', 'cilantro', 'citi', 'close', 'cocktail', 'coffe', 'come', 'comeback', 'comeher', '

,,10,1000,2,2star,3,5,5star,695,7th,...,wire,wish,wood,work,workshop,world,worth,ye,year,zerostar
0,0.023722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0795,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.069882,0.000000,0.000000,0.0795
1,0.093115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.00000,0.195412,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
2,0.048609,0.000000,0.162907,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
3,0.027549,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.092326,0.000000,0.00000,0.000000,0.00000,0.000000,0.081156,0.000000,0.000000,0.0000
4,0.042605,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
5,0.071840,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240764,0.0000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
6,0.041834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
7,0.046833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.00000,0.098285,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
8,0.088414,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
9,0.018280,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.00000,0.038362,0.00000,0.000000,0.000000,0.000000,0.061262,0.0000


In [47]:
from collections import defaultdict
dict_series = [0]*len(lines_)
i = 0
for rev in lines_:
    fq= defaultdict(int)
    for w in rev.split():
        fq[w] += 1
    dict_series[i] = fq
    i += 1

In [48]:
fq

defaultdict(int,
            {'-PRON-': 18,
             '-the': 3,
             '2': 1,
             '5_min': 1,
             '5_star': 1,
             'amazing': 1,
             'and\\/or': 1,
             'anymore': 1,
             'anything_else': 1,
             'astray': 1,
             'bar': 1,
             'base': 1,
             'big': 1,
             'blow': 1,
             'breakfast_burrito': 1,
             'cake': 1,
             'choice': 1,
             'coe': 1,
             'coffee': 1,
             'come': 1,
             'commuter': 1,
             'complaint': 1,
             'confection': 1,
             'cookie': 1,
             'crowd': 1,
             'cupcake': 2,
             'dangerous': 1,
             'dare': 1,
             'daunting': 1,
             'delicious': 1,
             'difficulty': 1,
             'dish': 1,
             'do_not': 4,
             'earth': 1,
             'equally': 1,
             'even_though': 1,
             'experience': 

In [49]:
# https://programminghistorian.org/en/lessons/counting-frequencies
dict_series[0:10]

[defaultdict(int,
             {"'s": 3,
              '-PRON-': 26,
              '7th': 1,
              'ask': 1,
              'ave': 1,
              'barber': 1,
              'barber_shop': 2,
              'barely': 1,
              'black': 2,
              'close': 1,
              'come': 1,
              'come_back': 1,
              'come_here': 1,
              'conclusion': 1,
              'cut': 2,
              'do_not': 1,
              'experience': 2,
              'extra': 1,
              'fade': 1,
              'go_back': 1,
              'guy': 3,
              'hair': 1,
              'haircut': 2,
              'hanlon': 1,
              'head': 1,
              'heavyset': 1,
              'imply': 1,
              'invoke': 1,
              'kind_of': 1,
              'left': 1,
              'length': 1,
              'lest': 1,
              'likely': 1,
              'little': 1,
              'loyal_customer': 1,
              'malice': 1,
            

In [50]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

review_series = [wordListToFreqDict(wordstring.split()) for wordstring in lines_]

In [51]:
review_series[0:5]

[{"'s": 3,
  '-PRON-': 26,
  '7th': 1,
  'ask': 1,
  'ave': 1,
  'barber': 1,
  'barber_shop': 2,
  'barely': 1,
  'black': 2,
  'close': 1,
  'come': 1,
  'come_back': 1,
  'come_here': 1,
  'conclusion': 1,
  'cut': 2,
  'do_not': 1,
  'experience': 2,
  'extra': 1,
  'fade': 1,
  'go_back': 1,
  'guy': 3,
  'hair': 1,
  'haircut': 2,
  'hanlon': 1,
  'head': 1,
  'heavyset': 1,
  'imply': 1,
  'invoke': 1,
  'kind_of': 1,
  'left': 1,
  'length': 1,
  'lest': 1,
  'likely': 1,
  'little': 1,
  'loyal_customer': 1,
  'malice': 1,
  'man': 1,
  'nate': 1,
  'occasion': 1,
  'place': 3,
  'presume': 2,
  'racism': 1,
  'razor': 1,
  'real': 1,
  'regular': 2,
  'remind': 1,
  'second': 2,
  'shape': 1,
  'simple': 2,
  'special': 1,
  'stay': 1,
  'steve': 1,
  'stupidity': 1,
  'style': 1,
  'tell': 1,
  'thank': 1,
  'think': 1,
  'tomorrow': 1,
  'walk_in': 1,
  'want': 2,
  'way': 1,
  'will_never': 1,
  'worth': 1,
  'zero_star': 1},
 {'-PRON-': 5,
  'business': 1,
  'know': 1,
  

In [52]:
# find a way to remove words which are not specific to restaurants
# idea: pick a business category which has very less similarity to 
        # restaurants and remove those words from our word lists

In [53]:
# Aggregating businesses by combined reviews will help in identifying similar restaurants.
    # can use this for recommending new restaurants
    # can also use this in identifying similar users (i.e similar restaurants liked by diff users)
# Aggregating user's by combining reviews will help in identifying similar restaurants.
    # we will get some sort of score
# probably could use that for users



In [54]:
### Spell checking
### english words has a max of two repeated letters consecutively
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

print(reduce_lengthening( "finallllllly" ))

finally


In [55]:
#MODULE = '/Users/rajeshpothamsetty/Downloads/pattern-master/pattern'
#import sys
#if MODULE not in sys.path: sys.path.append(MODULE)
#import pattern

from pattern3.en import spelling, suggest

word = "amazzziiing"
word_wlf = reduce_lengthening(word) #calling function defined above
print(word_wlf) #word lengthening isn't being able to fix it completely

correct_word = suggest(word_wlf) 
print(correct_word)

IndentationError: expected an indented block (tree.py, line 37)

In [56]:
from textblob import TextBlob
from textblob import Word

temp_textblob = TextBlob(x_temp)
#print(temp_textblob.correct())

In [ ]:
temp_correct = temp_textblob.correct()
temp_correct

In [57]:
clean_text(temp_correct) #removes 'e' from arrive; why?

NameError: name 'temp_correct' is not defined

In [58]:
TextBlob('reserv').correct()

TextBlob("reserve")

In [59]:
TextBlob('arriv').correct()

TextBlob("arrive")

In [60]:
## Pyspark Basics
## https://www.dezyre.com/apache-spark-tutorial/pyspark-tutorial

In [87]:
import findspark
findspark.init()

In [1]:
from operator import add
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml.feature import IDF
from pyspark.ml import Pipeline, PipelineModel

from pyspark.sql.functions import *
from pyspark.sql.types import *

import folium
import html

In [2]:
data_path = '/Volumes/Transcend/dataset/'
model_path = '/Volumes/Transcend/MDS_Yelp/model/'
outout_path = '/Volumes/Transcend/MDS_Yelp/output/'

In [3]:
business_df = spark.read.parquet(data_path + 'business-small.parquet')
business_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- business_name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [4]:
user_df = spark.read.parquet(data_path + 'user-small.parquet')
user_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- fans: long (nullable = true)
 |-- average_stars: double (nullable = true)



In [5]:
review_df = spark.read.parquet(data_path + 'review-small.parquet')
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



In [6]:
# create SQL view for later queries
review_df.createOrReplaceTempView("reviews")

# create review text dataframe
reviews_text = spark.sql("SELECT user_id, review_text FROM reviews")
reviews_text.show(3)

+--------------------+--------------------+
|             user_id|         review_text|
+--------------------+--------------------+
|u0LXt3Uea_GidxRW1...|Who would have gu...|
|u0LXt3Uea_GidxRW1...|Not bad!! Love th...|
|u0LXt3Uea_GidxRW1...|This is currently...|
+--------------------+--------------------+
only showing top 3 rows



In [9]:
# concatenate all reviews per restuarant

reviews_text_rdd = reviews_text.rdd
reviews_by_user_rdd = reviews_text_rdd.map(tuple).reduceByKey(add)  
reviews_by_user_df = spark.createDataFrame(reviews_by_user_rdd)
reviews_by_user_df = reviews_by_user_df \
                            .withColumnRenamed('_1', 'user_id') \
                            .withColumnRenamed('_2', 'text')
reviews_by_user_df.count()

73041

In [10]:
reviews_by_user_df.head()

Row(user_id='nOTl4aPC4tKHK35T3bNauQ', text='I give up on this place! I have gotten burritos and other things from this place on several occasions and it\'s always been sub-par. Bland food, overcooked, sometimes burnt (nachos). As a result, I haven\'t bothered going or ordering from there in well over a year. Well, today I had a craving for Mexican and since Toronto has a huge lack of Mexican restaurant choices and because I\'m busy working at home, my husband and myself decided to order from BarBurrito\'s delivery. I should also mention that if you go to justeat.ca BarBurrito charges $7 delivery. Orderit.ca has no charge. Don\'t know if that\'s the ordering website or the restaurant, but it\'s a tip for anyone who decides to order from this place.  So I went ahead and placed the order...only to have it be "rejected" by BarBurrito, because they are busy right now. This restaurant is listed on two different food ordering sites, but they won\'t deliver if they don\'t feel inclined?? SERIO

In [11]:
reviews_by_user_df.show(3)

+--------------------+--------------------+
|             user_id|                text|
+--------------------+--------------------+
|nOTl4aPC4tKHK35T3...|I give up on this...|
|Aj2IZibnWlSD1wWdq...|Love this place m...|
|6KM2pWFcsHYf4Nr9c...|Phenomenal brunch...|
+--------------------+--------------------+
only showing top 3 rows



In [72]:
## Example of using Word2vec
from pyspark.ml.feature import Word2Vec

# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.011087720049545169,-0.04713543206453324,-0.030747681856155396]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.02089989078896386,0.025788720164980204,-0.029980793063129695]

Text: [Logistic, regression, models, are, neat] => 
Vector: [0.03326573260128498,0.010403224825859071,-0.04281422272324562]



In [ ]:
## https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.Word2Vec

## The minimum number of times a token must appear to be included in the word2vec model's vocabulary"

Tuning of vectorSize and minCount required in word2vec model training.

In [79]:
%%time

# create text processing pipeline -- this a lengthy resource-intensive process
# Build the pipeline 
regexTokenizer = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'text', outputCol = 'token')
stopWordsRemover = StopWordsRemover(inputCol = 'token', outputCol = 'nostopwrd')
countVectorizer = CountVectorizer(inputCol="nostopwrd", outputCol="rawFeature")
iDF = IDF(inputCol="rawFeature", outputCol="idf_vec")
word2Vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'nostopwrd', outputCol = 'word_vec', seed=123)
vectorAssembler = VectorAssembler(inputCols=['idf_vec', 'word_vec'], outputCol='comb_vec')
pipeline = Pipeline(stages=[regexTokenizer, stopWordsRemover, countVectorizer, iDF, word2Vec, vectorAssembler])

# fit the model
pipeline_mdl = pipeline.fit(reviews_by_user_df)

#save the pipeline model
pipeline_mdl.write().overwrite().save(model_path + 'pipe_txt')

CPU times: user 123 ms, sys: 37.3 ms, total: 160 ms
Wall time: 9min 14s


In [7]:
# load the text transformation pipeline trained model
pipeline_mdl = PipelineModel.load(model_path + 'pipe_txt')

In [12]:
# transform the review data
reviews_by_user_trf_df = pipeline_mdl.transform(reviews_by_user_df)

In [13]:
# show the transformed review data
reviews_by_user_trf_df.select( 'text', 'nostopwrd', 'idf_vec', 'word_vec', 'comb_vec').show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|           nostopwrd|             idf_vec|            word_vec|            comb_vec|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|I give up on this...|[give, place, got...|(128365,[0,1,2,3,...|[-0.0374168294159...|(128465,[0,1,2,3,...|
|Love this place m...|[love, place, mak...|(128365,[0,1,2,3,...|[-0.0216792637887...|(128465,[0,1,2,3,...|
|Phenomenal brunch...|[phenomenal, brun...|(128365,[0,1,2,3,...|[-0.0473526970104...|(128465,[0,1,2,3,...|
|A born and bred T...|[born, bred, toro...|(128365,[0,1,2,3,...|[-0.0298230579082...|(128465,[0,1,2,3,...|
|We really enjoyed...|[really, enjoyed,...|(128365,[0,1,2,3,...|[-0.0115986177697...|(128465,[0,1,2,3,...|
|I was pretty unim...|[pretty, unimpres...|(128365,[0,1,2,3,...|[-0.0381316097274...|(128465,[0,1,2,3,...|
|I have never done...|[never, done, o

In [14]:
reviews_by_user_trf_df.select( 'nostopwrd').show(10)

+--------------------+
|           nostopwrd|
+--------------------+
|[give, place, got...|
|[love, place, mak...|
|[phenomenal, brun...|
|[born, bred, toro...|
|[really, enjoyed,...|
|[pretty, unimpres...|
|[never, done, one...|
|[seven, lives, se...|
|[best, indian, fo...|
|[good, ramen, bet...|
+--------------------+
only showing top 10 rows



In [15]:
reviews_by_user_trf_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nostopwrd: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeature: vector (nullable = true)
 |-- idf_vec: vector (nullable = true)
 |-- word_vec: vector (nullable = true)
 |-- comb_vec: vector (nullable = true)



In [16]:
type(iDF)

NameError: name 'iDF' is not defined

In [17]:
def CosineSim(vec1, vec2): 
    return np.dot(vec1, vec2) / np.sqrt(np.dot(vec1, vec1)) / np.sqrt(np.dot(vec2, vec2))

In [18]:
all_user_vecs = reviews_by_user_trf_df.select('user_id', 'word_vec').rdd.map(lambda x: (x[0], x[1])).collect()

In [19]:
type(all_user_vecs)

list

In [23]:
CosineSim(all_user_vecs[0][1], all_user_vecs[1][1])

0.8652645262685584

In [24]:
all_user_vecs[0]
# user_id = 'nOTl4aPC4tKHK35T3bNauQ'
# DenseVector() - vector respresentation of all the reviews of User

('nOTl4aPC4tKHK35T3bNauQ',
 DenseVector([-0.0374, -0.0827, -0.0607, 0.0706, 0.0262, -0.0358, -0.0004, -0.009, -0.0103, -0.0207, -0.0211, -0.0439, -0.0347, 0.0174, -0.0089, -0.0437, -0.0669, -0.0098, -0.0301, -0.0068, -0.0761, -0.0492, 0.0009, 0.0052, 0.0626, -0.0469, -0.0064, -0.0302, -0.0117, 0.0336, 0.0578, 0.0054, 0.0243, -0.0307, -0.0272, -0.0705, 0.0969, 0.0265, -0.0284, -0.0408, -0.0362, -0.0576, -0.0311, 0.0778, 0.0081, -0.0428, -0.048, 0.0279, -0.0118, 0.014, 0.0095, -0.0122, -0.0123, -0.0328, -0.0584, 0.0126, 0.0096, 0.0456, 0.0413, 0.0209, -0.0138, -0.0004, 0.0237, -0.0566, -0.0347, 0.0297, 0.0447, 0.0419, -0.0252, 0.0758, 0.0014, -0.0045, 0.0094, 0.0375, 0.0088, -0.0188, -0.0655, 0.0098, -0.0166, 0.0423, 0.001, -0.0137, -0.0024, -0.0112, -0.0131, -0.0028, 0.0135, 0.0187, -0.0107, 0.0014, -0.0016, -0.0062, -0.0262, 0.0269, -0.0136, -0.0092, -0.0441, 0.0108, -0.0502, 0.025]))

In [ ]:
all_user_vecs[30]

In [ ]:
# To do consider only if the USer had minm. number of reviews
# Tuning VectorSize and minCount
# maybe removing non-restaurant related words...

In [25]:
def getSimilarUsers(b_ids, sim_user_limit=10):
    
    schema = StructType([   
                            StructField("user_id", StringType(), True)
                            ,StructField("score", IntegerType(), True)
                            ,StructField("input_user_id", StringType(), True)
                        ])
    
    similar_user_df = spark.createDataFrame([], schema)
    
    for b_id in b_ids:
        
        input_vec = [(r[1]) for r in all_user_vecs if r[0] == b_id][0]
        
        similar_user_rdd = sc.parallelize((i[0], float(CosineSim(input_vec, i[1]))) for i in all_user_vecs)
        
        similar_user_df = spark.createDataFrame(similar_user_rdd) \
            .withColumnRenamed('_1', 'user_id') \
            .withColumnRenamed('_2', 'score') \
            .orderBy("score", ascending = False)
            
        similar_user_df = similar_user_df.dropna()    
        similar_user_df = similar_user_df.filter(col("user_id") != b_id).limit(sim_user_limit)
        similar_user_df = similar_user_df.withColumn('input_user_id', lit(b_id))
        
        similar_user_df = similar_user_df \
                                    .union(similar_user_df)
        
    
    return similar_user_df

In [26]:
def getUserDetails(in_bus):
    
    a = in_bus.alias("a")
    b = user_df.alias("b")
    
    return a.join(b, col("a.user_id") == col("b.user_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.user_id'), col('b.user_name'),col('b.review_count')])

In [27]:
# test with two restaurants

uids = ['nOTl4aPC4tKHK35T3bNauQ', 'QBac9-Ii6jR-yLsQ5MVTHg']

print('\ninput user details:')
user_df.select('user_id','user_name', 'review_count') \
    .filter(user_df.user_id.isin(uids) == True).show(truncate=False)
    
# get top 10 similar business
sim_users = getUserDetails(getSimilarUsers(uids))

print('Top 10 similar Users for each input restaurant are:"')
sim_users.select('input_user_id', 'a.user_id', 'user_name', 'score','review_count').toPandas()


input user details:
+----------------------+---------+------------+
|user_id               |user_name|review_count|
+----------------------+---------+------------+
|nOTl4aPC4tKHK35T3bNauQ|Katherine|148         |
|QBac9-Ii6jR-yLsQ5MVTHg|Alex     |13          |
+----------------------+---------+------------+



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Top 10 similar Users for each input restaurant are:"


,input_user_id,user_id,user_name,score,review_count
0,QBac9-Ii6jR-yLsQ5MVTHg,MpN81tQOL86GaFse-_tTRQ,Amy,0.913263,46
1,QBac9-Ii6jR-yLsQ5MVTHg,MpN81tQOL86GaFse-_tTRQ,Amy,0.913263,46
2,QBac9-Ii6jR-yLsQ5MVTHg,bPUpO-bP6BmAGvSwPyDsng,Michael,0.905848,142
3,QBac9-Ii6jR-yLsQ5MVTHg,bPUpO-bP6BmAGvSwPyDsng,Michael,0.905848,142
4,QBac9-Ii6jR-yLsQ5MVTHg,zkKv10FEw-iWJBHvK0mGfA,Paul,0.903733,55
5,QBac9-Ii6jR-yLsQ5MVTHg,zkKv10FEw-iWJBHvK0mGfA,Paul,0.903733,55
6,QBac9-Ii6jR-yLsQ5MVTHg,eV5usRjY2cDqNKVv8wXroA,Sam,0.905077,105
7,QBac9-Ii6jR-yLsQ5MVTHg,eV5usRjY2cDqNKVv8wXroA,Sam,0.905077,105
8,QBac9-Ii6jR-yLsQ5MVTHg,xcBcIMVWEx1p7V8ACaeHpQ,Rick,0.918668,83
9,QBac9-Ii6jR-yLsQ5MVTHg,xcBcIMVWEx1p7V8ACaeHpQ,Rick,0.918668,83


In [ ]:
getSimilarUsers(uids)

In [ ]:
getSimilarUsers(uids).head(20)

In [ ]:
schema = StructType([   
                        StructField("user_id", StringType(), True)
                        ,StructField("score", IntegerType(), True)
                        ,StructField("input_user_id", StringType(), True)
                    ])

similar_user_df = spark.createDataFrame([], schema)

In [ ]:
sim_user_limit = 10

for b_id in uids:
    print(b_id)
    input_vec = [(r[1]) for r in all_user_vecs if r[0] == b_id][0]
    #input_vec = reviews_by_business_trf_df.select('word_vec')\
                #.filter(reviews_by_business_trf_df['business_id'] == b_id)\
                #.collect()[0][0]

    similar_user_rdd = sc.parallelize((i[0], float(CosineSim(input_vec, i[1]))) for i in all_user_vecs)

    similar_user_df = spark.createDataFrame(similar_user_rdd) \
        .withColumnRenamed('_1', 'user_id') \
        .withColumnRenamed('_2', 'score') \
        .orderBy("score", ascending = False)
    similar_user_df = similar_user_df.dropna()
    #.where(isnan(col("a")))
    similar_user_df = similar_user_df.filter(col("user_id") != b_id).limit(sim_user_limit)
    similar_user_df = similar_user_df.withColumn('input_user_id', lit(b_id))

    similar_user_df = similar_user_df \
                                .union(similar_user_df)

    print("-------------------------------------")
    print(similar_user_df.take(10))
    print("-------------------------------------")

In [ ]:
similar_user_rdd

In [ ]:
print(similar_user_df.take(100))